# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120312e+02     1.542890e+00
 * time: 0.10279703140258789
     1     1.046852e+01     9.368601e-01
 * time: 1.3731598854064941
     2    -1.182795e+01     1.021729e+00
 * time: 1.4390039443969727
     3    -3.394927e+01     7.683734e-01
 * time: 1.5478858947753906
     4    -4.739233e+01     5.946328e-01
 * time: 1.6200058460235596
     5    -5.682970e+01     2.099278e-01
 * time: 1.7127668857574463
     6    -5.977248e+01     1.176457e-01
 * time: 1.78489089012146
     7    -6.085636e+01     4.966352e-02
 * time: 1.8446638584136963
     8    -6.126127e+01     5.988999e-02
 * time: 1.9057650566101074
     9    -6.155950e+01     3.586043e-02
 * time: 1.9878590106964111
    10    -6.178565e+01     3.153324e-02
 * time: 2.0483219623565674
    11    -6.196417e+01     2.154083e-02
 * time: 2.1134049892425537
    12    -6.203174e+01     1.770439e-02
 * time: 2.1683359146118164
    13    -6.210578e+01     1.450740e-02
 * time: 2.23113894462

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557695
    AtomicNonlocal      14.8522653
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336823

    total               -62.261666460951
